In [72]:
from langchain_openai import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_core.output_parsers import StrOutputParser
from langchain_community.retrievers import WikipediaRetriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.runnables import RunnablePassthrough
import os

In [73]:
# SECURE THIS KEY!
os.environ["AZURE_OPENAI_API_KEY"] = ""
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://azqoreai.openai.azure.com/"
os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] = "gpt-35-turbo-16k_azqore_ai"
os.environ["AZURE_OPENAI_API_VERSION"] = "2024-02-01"

In [74]:
model = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
)
output_parser= StrOutputParser()

#this retriever (more on this later) let us access Wikipedia
retriever = WikipediaRetriever()

In [75]:


prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a highly efficient assistant that excels at creating concise and comprehensive summaries."),
    ("human", "Create a summary in 10 lines about this topic {input} using only the data provided here : {context}")
])



#this chain will invoke the llm passing the documents retrieved from from wikipedia as input in the "context" variable
llm_chain = create_stuff_documents_chain(model, prompt_template)

#this chain will invoke the wikipedia retriever then pass the result to llm_chain
chain = create_retrieval_chain(retriever, llm_chain)


output = chain.invoke({
    "input": "euro 2024",
})
output_parser.invoke(output['answer'])



'The 2024 UEFA European Football Championship, also known as Euro 2024, is the 17th edition of the tournament and is being held in Germany from 14 June to 14 July 2024. The tournament features 24 teams, including Georgia making their debut in the European Championship finals. This is the third time matches are played on German territory, with Leipzig hosting for the first time in what was formerly East Germany. Germany is the solo host nation for the first time since the 2006 FIFA World Cup. The tournament returned to its regular four-year cycle after being postponed in 2020 due to the COVID-19 pandemic. Italy, the defending champions, were eliminated in the round of 16 by Switzerland. The tournament follows a knockout format, with extra time and penalty shootouts used to determine winners in case of a tie. The final will be held at Olympiastadion in Berlin. The qualifying tournament for Euro 2024 took place from March 2023 to March 2024, with the top two teams from each group and the 

In [76]:

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a highly efficient assistant that excels at creating concise and comprehensive summaries."),
    ("human", "Create a summary in 10 lines about this topic {input} using only the data provided here : {context}")
])

#this chain will invoke the llm passing the documents retrieved from from wikipedia as input in the "context" variable
chain = create_retrieval_chain(retriever, prompt_template)

output = chain.invoke({
    "input": "euro 2024",
})
output["context"]

[Document(metadata={'title': 'UEFA Euro 2024', 'summary': "The 2024 UEFA European Football Championship, commonly referred to as UEFA Euro 2024 (stylised as UEFA EURO 2024), or simply Euro 2024, is the ongoing 17th edition of the UEFA European Championship, the quadrennial international football  championship organised by UEFA for the European men's national teams of its member associations. Germany is hosting the tournament, which is taking place from 14 June to 14 July 2024. The tournament comprises 24 teams, with Georgia being the only team making their European Championship finals debut. \nIt is the third time that European Championship matches are played on German territory and the second time in reunified Germany, as West Germany hosted the tournament's 1988 edition, and four matches of the multi-national Euro 2020 were played in Munich. It is the first time the competition is held in what was formerly East Germany, with Leipzig as a host city, as well as the first major tourname

In [77]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a highly efficient assistant that excels at creating concise and comprehensive summaries."),
    ("human", "Create a summary in 10 lines about this topic {input} using only the data provided here : {context}")
])

chain = (
    {"context": retriever, "input": RunnablePassthrough()}
    | prompt_template
    | model
    | output_parser
)

chain.invoke("euro 2024")


'UEFA Euro 2024 is the 17th edition of the UEFA European Championship, hosted by Germany from 14 June to 14 July 2024. The tournament features 24 teams, with Georgia making their European Championship finals debut. This is the third time European Championship matches are played in Germany and the first time in what was formerly East Germany. Leipzig is one of the host cities. It is the first major tournament since the 2006 FIFA World Cup that Germany serves as a solo host nation. Italy, the defending champions, were eliminated in the round of 16 by Switzerland. The knockout stage of Euro 2024 began on 29 June 2024 and will end with the final at Olympiastadion in Berlin on 14 July 2024. The qualifying tournament for Euro 2024 was played from March 2023 to March 2024, with 23 teams joining host Germany in the final tournament. The competition was linked with the 2022–23 UEFA Nations League.'

In [79]:
prompt_template.input_variables

['context', 'input']

In [82]:
model.output_schema.schema_json()

'{"title": "AzureChatOpenAIOutput", "anyOf": [{"$ref": "#/definitions/AIMessage"}, {"$ref": "#/definitions/HumanMessage"}, {"$ref": "#/definitions/ChatMessage"}, {"$ref": "#/definitions/SystemMessage"}, {"$ref": "#/definitions/FunctionMessage"}, {"$ref": "#/definitions/ToolMessage"}], "definitions": {"ToolCall": {"title": "ToolCall", "type": "object", "properties": {"name": {"title": "Name", "type": "string"}, "args": {"title": "Args", "type": "object"}, "id": {"title": "Id", "type": "string"}}, "required": ["name", "args", "id"]}, "InvalidToolCall": {"title": "InvalidToolCall", "type": "object", "properties": {"name": {"title": "Name", "type": "string"}, "args": {"title": "Args", "type": "string"}, "id": {"title": "Id", "type": "string"}, "error": {"title": "Error", "type": "string"}}, "required": ["name", "args", "id", "error"]}, "UsageMetadata": {"title": "UsageMetadata", "type": "object", "properties": {"input_tokens": {"title": "Input Tokens", "type": "integer"}, "output_tokens": 